## Import necessary libraries

In [6]:
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors


## Function to create user-item matrix

In [7]:
def create_matrix(df):
    N = len(df['userId'].unique())
    M = len(df['movieId'].unique())
    
    # Map Ids to indices
    user_mapper = dict(zip(np.unique(df["userId"]), list(range(N))))
    movie_mapper = dict(zip(np.unique(df["movieId"]), list(range(M))))
    
    # Map indices to IDs
    user_inv_mapper = dict(zip(list(range(N)), np.unique(df["userId"])))
    movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["movieId"])))
    
    user_index = [user_mapper[i] for i in df['userId']]
    movie_index = [movie_mapper[i] for i in df['movieId']]
    
    X = csr_matrix((df["rating"], (movie_index, user_index)), shape=(M, N))
    return X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper

## Function to find similar movies using KNN

In [9]:
def find_similar_movies(movie_id, X, k, metric='cosine', show_distance=False):
    neighbour_ids = []
    movie_ind = movie_mapper[movie_id]
    movie_vec = X[movie_ind]
    k+=1
    kNN = NearestNeighbors(n_neighbors=k, algorithm="brute", metric=metric)
    kNN.fit(X)
    movie_vec = movie_vec.reshape(1,-1)
    neighbour = kNN.kneighbors(movie_vec, return_distance=show_distance)
    for i in range(0,k):
        n = neighbour.item(i)
        neighbour_ids.append(movie_inv_mapper[n])
    neighbour_ids.pop(0)
    return neighbour_ids

## Function to recommend movies for a user based on their preferences

In [20]:
def recommend_movies_for_movie(movie_title):  # Added function to recommend movies based on user-input movie title
    movie = movies[movies['title'].str.contains(movie_title, case=False)]  # Search movies containing the user-input movie title
    if movie.empty:
        print(f"No movies found containing '{movie_title}'. Please try again.")  # Display error message if no matching movie found
        return
    
    movie_id = movie.iloc[0]['movieId']
    movie_titles = dict(zip(movies['movieId'], movies['title']))
    similar_ids = find_similar_movies(movie_id, X, k=10)
    movie_title = movie_titles.get(movie_id, "Movie not found")
    if movie_title == "Movie not found":
        print(f"Movie with ID {movie_id} not found.")
        return

    recommendation_message = f"Since you watched {movie_title}, you might also like:\n"
    for i in similar_ids:
        recommendation_message += movie_titles.get(i, "Movie not found") + "\n"
    
    print(recommendation_message)

## Load the datasets

In [ ]:
ratings = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/ratings.csv")
movies = pd.read_csv("https://s3-us-west-2.amazonaws.com/recommender-tutorial/movies.csv")

## Create user-item matrix

In [ ]:
X, user_mapper, movie_mapper, user_inv_mapper, movie_inv_mapper = create_matrix(ratings)

## Recommend movies for a specific user

In [ ]:
movie_title = input("Enter a movie title: ")  # Prompt user to input a movie title
recommend_movies_for_movie(movie_title)  # Call the function to recommend movies based on user-input movie title